In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import ast

import sys
print('sys.executable =', sys.executable)
import importlib
print('keybert spec =', importlib.util.find_spec('keybert'))
import seaborn as sns
from keybert import KeyBERT

# Load dataset
df = pd.read_csv("Uitgebreide_VKM_dataset.csv")

sys.executable = C:\Users\jeroe\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
keybert spec = ModuleSpec(name='keybert', loader=<_frozen_importlib_external.SourceFileLoader object at 0x000002A3348A0C90>, origin='C:\\Users\\jeroe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\keybert\\__init__.py', submodule_search_locations=['C:\\Users\\jeroe\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\keybert'])


In [6]:
df.columns = df.columns.str.strip()
print(df.columns)

Index(['id', 'name', 'shortdescription', 'description', 'content',
       'studycredit', 'location', 'contact_id', 'level', 'learningoutcomes',
       'Rood', 'Groen', 'Blauw', 'Geel', 'module_tags',
       'interests_match_score', 'popularity_score', 'estimated_difficulty',
       'available_spots', 'start_date'],
      dtype='object')


In [6]:
# Make a copy of the DataFrame
df_copy = df.copy()

# remove the color columns if
colorCols = ['Rood', 'Geel', 'Blauw', 'Groen']
if([col for col in colorCols if col in df_copy.columns]):
    df_copy = df_copy.drop(columns=colorCols)

# Delete rows where id is 293, 294, 295
ids_to_remove = [293, 294, 295]
df_copy = df_copy[~df_copy['id'].isin(ids_to_remove)]

# Update short_description
def update_short_description(row):
    desc = str(row.get('description', '')).strip()
    cont = str(row.get('content', '')).strip()
    short = str(row.get('shortdescription', '')).strip()

    # Only change when short_description == 'Ntb'
    if short.lower() == 'ntb' or short.lower() == 'nan':
        # If description or content is also 'ntb', keep 'Ntb'
        if desc.lower() == 'ntb' or cont.lower() == 'ntb':
            return 'Ntb'
        else:
            return f"{desc} {cont}".strip()
    return short

df_copy['shortdescription'] = df_copy.apply(update_short_description, axis=1)


# --- Clean up the learningoutcomes column ---
df_copy['learningoutcomes'] = df_copy['learningoutcomes'].replace(
    {
        r'(?i)^(ntb|n\.t\.b\.|n\.n\.b\.|volgt|nog nader te bepalen|nog te formuleren|nog niet bekend|nog te bepalen|\s*)$':
        'Nog te bepalen'
    },
    regex=True
)

# Also handle NaN (missing) values
df_copy['learningoutcomes'] = df_copy['learningoutcomes'].fillna('Nog te bepalen')

def is_empty_tag(value):
    """Check of een tag waarde leeg of ntb is"""
    # Check voor None/NaN (maar niet voor arrays)
    if value is None or (isinstance(value, float) and np.isnan(value)):
        return True
    
    if isinstance(value, str):
        v = value.strip()
        if v == "" or v == "[]" or v.lower() in ["['ntb']", "ntb"]:
            return True
        
        # Probeer te parsen als lijst
        try:
            parsed = ast.literal_eval(v)
            if isinstance(parsed, list):
                if len(parsed) == 0:
                    return True
                if len(parsed) == 1 and str(parsed[0]).lower() == "ntb":
                    return True
        except:
            pass
    
    if isinstance(value, list):
        if len(value) == 0:
            return True
        if len(value) == 1 and str(value[0]).lower() == "ntb":
            return True
    
    return False

def extract_keywords_from_text(text, model):
    """Extraheer keywords uit tekst met KeyBERT"""
    if not isinstance(text, str) or not text.strip():
        return []
    
    try:
        keywords = model.extract_keywords(
            text, 
            keyphrase_ngram_range=(1, 2)
        )
        return [keyword[0] for keyword in keywords]
    except:
        return []

# Check of er lege tags zijn die gevuld moeten worden
if 'module_tags' in df_copy.columns:
    # Veiligere manier om empty mask te maken
    empty_mask = []
    for idx, value in df_copy['module_tags'].items():
        empty_mask.append(is_empty_tag(value))
    
    empty_mask = pd.Series(empty_mask, index=df_copy.index)
    rows_to_process = empty_mask.sum()
    
    if rows_to_process > 0:
        print(f"Gevonden {rows_to_process} rijen met lege/ontbrekende tags")
        print("Start keyword extractie...")
        
        # Initialiseer KeyBERT model (alleen als nodig)
        kw_model = KeyBERT()
        
        # Maak combined text kolom (alleen als niet bestaat)
        if 'combined_text' not in df_copy.columns:
            df_copy['combined_text'] = (
                df_copy['description'].fillna('') + " " + 
                df_copy['content'].fillna('')
            )
        
        # Genereer nieuwe tags alleen voor lege rijen
        def generate_tags_if_empty(row):
            if is_empty_tag(row['module_tags']):
                return extract_keywords_from_text(row['combined_text'], kw_model)
            else:
                # Bestaande tags behouden, converteer string naar lijst indien nodig
                current = row['module_tags']
                if isinstance(current, str):
                    try:
                        return ast.literal_eval(current)
                    except:
                        return [current]
                return current
        
        # Update alleen de lege tags
        df_copy['module_tags'] = df_copy.apply(generate_tags_if_empty, axis=1)
        
        print(f"Tags gegenereerd voor {rows_to_process} modules")
        
        # Toon resultaten
        updated_rows = df_copy[empty_mask][['name', 'module_tags']].head(5)
        print("\nVoorbeelden van gegenereerde tags:")
        for idx, row in updated_rows.iterrows():
            print(f"- {row['name']}: {row['module_tags']}")
    else:
        print("Alle modules hebben al tags - geen actie nodig")
else:
    print("Kolom 'module_tags' niet gevonden in dataset")

# create new csv
df_copy.to_csv("Uitgebreide_VKM_dataset_cleaned.csv", index=False)

# Verify the change
display(df_copy)


Gevonden 28 rijen met lege/ontbrekende tags
Start keyword extractie...
Tags gegenereerd voor 28 modules

Voorbeelden van gegenereerde tags:
- Oncologie: ['oncologische ziektebeelden', 'oncologische zorg', 'naasten oncologie', 'oncologische', 'zorgvragers en']
- Pro-active nursing: ['active nursing', 'nursing', 'een professionele', 'nursing leer', 'professionele beoordeling']
- Jongeren en actuele problematiek: ['problematiek te', 'actuele problematiek', 'en huiselijk', 'huiselijk geweld', 'waar het']
- Management in de zorg: ['kwaliteitsmanagement zorgsector', 'en kwaliteitsmanagement', 'management kwaliteitsmanagement', 'kwaliteitsmanagement en', 'management zorg']
- Gedrag: ['verbinding van', 'trainingen sterk', 'leerkracht verbinding', 'gedrag van', 'orthopedagogiek trainingen']


,id,name,shortdescription,description,content,studycredit,location,contact_id,level,learningoutcomes,module_tags,interests_match_score,popularity_score,estimated_difficulty,available_spots,start_date,combined_text
0,159,Kennismaking met Psychologie,"Brein, gedragsbeinvloeding, ontwikkelingspsych...",In deze module leer je hoe je gedrag van jezel...,In deze module leer je hoe je gedrag van jezel...,15,Den Bosch,58,NLQF5,A. Je beantwoordt vragen in een meerkeuze kenn...,"[brein, gedragsbeinvloeding, ontwikkelingspsyc...",0.54,319,1,79,2025-12-24,In deze module leer je hoe je gedrag van jezel...
1,160,Learning and working abroad,"Internationaal, persoonlijke ontwikkeling, ver...",Studenten kiezen binnen de (stam) van de oplei...,Studenten kiezen binnen de (stam) van de oplei...,15,Den Bosch,58,NLQF5,De student toont professioneel gedrag conform ...,"[internationaal, persoonlijke, ontwikkeling, v...",0.92,172,5,56,2025-12-20,Studenten kiezen binnen de (stam) van de oplei...
2,161,Proactieve zorgplanning,"Proactieve zorgplanning, cocreatie, ziekenhuis",Het Jeroen Bosch ziekenhuis wil graag samen me...,Het Jeroen Bosch ziekenhuis wil graag samen me...,15,Den Bosch,59,NLQF5,De student past pro actieve zorgplanning toe b...,"[proactieve, zorgplanning, cocreatie, ziekenhuis]",0.78,217,5,55,2025-09-23,Het Jeroen Bosch ziekenhuis wil graag samen me...
3,162,Rouw en verlies,"Rouw & verlies, palliatieve zorg & redeneren, ...",In deze module wordt stil gestaan bij rouw en ...,In deze module wordt stil gestaan bij rouw en ...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"[rouw, verlies, palliatieve, zorg, redeneren, ...",0.69,454,1,54,2025-10-25,In deze module wordt stil gestaan bij rouw en ...
4,163,Acuut complexe zorg,"Acute zorg, complexiteit, ziekenhuis, revalidatie",In deze module kunnen studenten zich verdiepen...,In deze module kunnen studenten zich verdiepen...,30,Den Bosch,58,NLQF6,De student regisseert en voert (deels) zelfsta...,"[acute, zorg, complexiteit, ziekenhuis, revali...",0.40,178,5,38,2025-11-19,In deze module kunnen studenten zich verdiepen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,393,Ethiek & kritisch denken,"ethiek, filosofie, kritisch denken, ethisch ha...",Ethiek en ethisch handelen wordt steeds belang...,Ethiek en ethisch handelen wordt steeds belang...,30,Breda en Den Bosch,124,NLQF6,De student demonstreert persoonlijke groei op ...,"[ethiek, filosofie, kritisch, denken, ethisch,...",0.37,248,5,32,2025-10-17,Ethiek en ethisch handelen wordt steeds belang...
207,394,Avans Innovative Studio Junior,"persoonlijke ontwikkeling, interdisciplinair, ...",Je leert hoe welke waarde jouw werk heeft. Hoe...,Je leert hoe welke waarde jouw werk heeft. Hoe...,15,Breda,92,NLQF6,De student demonstreert persoonlijke groei op ...,"[persoonlijke, ontwikkeling, interdisciplinair...",0.73,299,5,62,2025-09-01,Je leert hoe welke waarde jouw werk heeft. Hoe...
208,395,Avans Innovative Studio Senior,Ook heb je de mogelijkheid om met creatieve on...,Ook heb je de mogelijkheid om met creatieve on...,Ook heb je de mogelijkheid om met creatieve on...,30,Breda,92,NLQF6,De student demonstreert persoonlijke groei op ...,"[mogelijkheid om, mogelijkheid, het werkveld, ...",0.30,105,5,62,2025-11-29,Ook heb je de mogelijkheid om met creatieve on...
209,396,Succesvol managen van een project,"Project Evaluatie, Scrum, Planning, Agile, Pri...","Leren om succesvol een project (innovatie, cha...","Leren om succesvol een project (innovatie, cha...",15,Breda,125,NLQF5,"1) Inzicht/kennis in de diverse factoren, zoal...","[project, evaluatie, scrum, planning, agile, p...",0.40,135,5,67,2025-09-14,"Leren om succesvol een project (innovatie, cha..."
